In [59]:
#Task 21 2m^6 + 5m^6 - 8m^6 - 11m^6
class Operation():

    def __init__(self, input_nodes = []):
        "Intialize an Operation"
        self.input_nodes = input_nodes
        self.output_nodes = []

        for node in input_nodes:
            node.output_nodes.append(self)

        _default_graph.operations.append(self)

    def compute(self):

        pass

In [60]:
class add(Operation):

    def __init__(self, x, y):

        super().__init__([x, y])

    def compute(self, x_var, y_var):

        self.inputs = [x_var, y_var]
        return x_var + y_var

class multiply(Operation):

    def __init__(self, a, b):

        super().__init__([a, b])

    def compute(self, a_var, b_var):

        self.inputs = [a_var, b_var]
        return a_var * b_var

class Placeholder():

    def __init__(self):

        self.output_nodes = []

        _default_graph.placeholders.append(self)

class Variable():

    def __init__(self, initial_value = None):

        self.value = initial_value
        self.output_nodes = []

        _default_graph.variables.append(self)
        

In [61]:
class Graph():

    def __init__(self):

        self.operations = []
        self.placeholders = []
        self.variables = []

    def set_as_default(self):
        "Sets this Graph instance as the Global Default Graph"
        global _default_graph
        _default_graph = self

In [62]:
g = Graph()
g.set_as_default()
a = Variable(2)
b = Variable(5)
c = Variable(-8)
d = Variable(-11)

m = Placeholder()

m1 = multiply(m,m) #m^2
m2 = multiply(m1,m) #m^3
m3 = multiply(m2,m) #m^4
m4 = multiply(m3,m) #m^5
m5 = multiply(m4,m) #m^6

e = multiply(a,m5) #2m^6
f = multiply(b,m5) #5m^6
g = add(e,f) #2m^6 + 5m^6
h = multiply(c,m5) #-8m^6
i = add(g,h) #2m^6 + 5m^6 - 8m^6
j = multiply(d,m5) #- 11m^6
k = add(i, j) #2m^6 + 5m^6 - 8m^6  - 11m^6


In [63]:
import numpy as np

def traverse_postorder(operation):
    nodes_postorder = []
    def recurse(node):
        if isinstance(node, Operation):
            for input_node in node.input_nodes:
                recurse(input_node)
        nodes_postorder.append(node)

    recurse(operation)
    return nodes_postorder

class Session:

    def run(self, operation, feed_dict = {}):
        "operation: The operation to compute"
        "feed_dict: Dictionary mapping placeholders to input values (the data)"

        nodes_postorder = traverse_postorder(operation)

        for node in nodes_postorder:

            if type(node) == Placeholder:

                node.output = feed_dict[node]

            elif type(node) == Variable:

                node.output = node.value

            else: # Operation

                node.inputs = [input_node.output for input_node in node.input_nodes]


                node.output = node.compute(*node.inputs)

            if type(node.output) == list:
                node.output = np.array(node.output)

        return operation.output


In [64]:
sess = Session()

result = sess.run(operation=k,feed_dict={m:[1, -1, 10, 15]})


In [65]:
result

array([       -12,        -12,  -12000000, -136687500])

In [66]:
2*(1*1*1*1*1*1) + 5*(1*1*1*1*1*1) - 8*(1*1*1*1*1*1) - 11*(1*1*1*1*1*1)

-12

In [67]:
2*((-1)*(-1)*(-1)*(-1)*(-1)*(-1)) + 5*((-1)*(-1)*(-1)*(-1)*(-1)*(-1)) - 8*((-1)*(-1)*(-1)*(-1)*(-1)*(-1)) - 11*((-1)*(-1)*(-1)*(-1)*(-1)*(-1))

-12

In [68]:
2*(10*10*10*10*10*10) + 5*(10*10*10*10*10*10) - 8*(10*10*10*10*10*10) - 11*(10*10*10*10*10*10)

-12000000

In [57]:
2*(15*15*15*15*15*15) + 5*(15*15*15*15*15*15) - 8*(15*15*15*15*15*15) - 11*(15*15*15*15*15*15)

-136687500